In [1]:
import os
import re
import time
import math
import torch
import random
import pickle
import os.path
import torchtext
import matplotlib
import numpy as np
import pandas as pd
import torch.nn as nn
import sentencepiece as spm


from glob import glob
from torch import optim
from itertools import cycle
from torchtext import vocab
from konlpy.tag import Mecab
from torchinfo import summary
from torch.nn import Transformer
from torch.utils.data import DataLoader
from timeit import default_timer as timer
from soyspacing.countbase import CountSpace
from torch.nn.utils.rnn import pad_sequence
from nltk.translate.bleu_score import sentence_bleu
from torchtext.vocab import build_vocab_from_iterator


mecab = Mecab()

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)


In [3]:
os.chdir("/data/kmkim/chatbot_data_model")
with open('data_sample.pkl', 'rb') as f:
    data_load = pickle.load(f)
with open('renew_dent_1000_hong.pkl', 'rb') as f: ### 새로운 unifiy 데이터를 여기에 넣어야함
    sample_load2 = pickle.load(f)

In [4]:
data_load

[['환자분 성함과 생년월일 부탁드립니다', 'ㅇ님이요.'],
 ['생년월일은요?', '잠시만요. ㅇ년 ㅇ월 ㅇ일입니다.'],
 ['ㅇ년 ㅇ월 ㅇ일이요?', '네 맞습니다.'],
 ['보고 내역 관리에서 검사 상태가 의뢰 중이 아니라는 말씀이시죠?', '네 맞습니다.'],
 ['잠시만요', '네.'],
 ['선생님 이 부분은 저희 시스템 오류가 맞고요', '네 그렇군요.'],
 ['저희가 시스템적으로 처리하고 연락드리겠습니다', '네 그렇게 해주세요.'],
 ['선생님 연락처와 성함 부탁드립니다', '저는 ㅇ이고요.'],
 ['전화번호는요?', 'ㅇ ㅇ ㅇ번입니다.'],
 ['저희가 처리하고 연락드리겠습니다', '네 감사합니다.'],
 ['선생님 확인해보겠습니다 잠시만 기다려주세요', '네 알겠습니다.'],
 ['선생님 위험군 메뉴얼을 선택하기 위해서는 전수 감시 유저 권한이 필요합니다', '아 그렇군요.'],
 ['안내해드리겠습니다 지금 접속 중이신가요?', '네 들어와있습니다.'],
 ['맞습니다 선택하시면 조그만 모 안에 승인 단체 조회라고 보이시나요?', '네 있어요.'],
 ['그러면 승인을 클릭하시면 신청 가능으로 되실 거예요', '네 신청 가능이요.'],
 ['이제 단체 선택해서 ㅇ 지원 선택해서 조회해 주세요', '네 알겠습니다.'],
 ['이제 전수 감시 유저 신청해 주시면 됩니다', '네 했습니다.'],
 ['승인 처리 완료했습니다 선생님 확인해보시면 되겠습니다', '네 감사합니다.'],
 ['로그아웃하셨다가 다시 로그인해서 진행해보십시오', '네 해볼게요.'],
 ['더 문의 사항 없으시면 통화 종료 하겠습니다', '네 수고하세요.'],
 ['환자분 주소이신가요?', '네 이 사이트에서 검색을 하면 안 나와요.'],
 ['주소 불러 주세요', 'ㅇ시 ㅇ구 ㅇ동 ㅇ요.'],
 ['선생님 구도로 주소로는 안되시고요 도로명 주소로 해야 되십니다', 'ㅇ로 ㅇ길 ㅇ요.'],
 ['그 주소로 조회해보시겠어요?', '이거로 해도 안 나와요.'],
 [

In [5]:
sample_load2

[['안녕하세요, 성함이 어떻게 되세요?', '홍길동입니다.'],
 ['안녕하세요, 성함이요?', '홍길동입니다.'],
 ['안녕하세요, 이름이 뭔가요?', '홍길동입니다.'],
 ['이름이 어떻게 되세요?', '홍길동입니다.'],
 ['환자분 성함이요?', '홍길동입니다.'],
 ['환자분 성함 말씀 해 주세요.', '홍길동입니다.'],
 ['성함이 어떻게 되실까요?', '홍길동입니다.'],
 ['환자분 이름은요?', '홍길동입니다.'],
 ['이름 말씀 해 주세요.', '홍길동입니다.'],
 ['이름이 어떻게 되시죠?', '홍길동입니다.'],
 ['성함이 어떻게 되시죠?', '홍길동입니다.'],
 ['안녕하세요, 이름이요?', '홍길동입니다.'],
 ['성함 말씀 해 주세요.', '홍길동입니다.'],
 ['이름 좀 말씀 해 주세요. ', '홍길동입니다.'],
 ['이름이 어떻게 되실까요?', '홍길동입니다.'],
 ['이름이 무엇이죠?', '홍길동입니다.'],
 ['환자분 이름이요?', '홍길동입니다.'],
 ['환자분 성함이 어떻게 되세요?', '홍길동입니다.'],
 ['안녕하세요, 성함 말씀 해 주세요.', '홍길동입니다.'],
 ['안녕하세요, 이름 말씀 해주세요.', '홍길동입니다.'],
 ['환자분 이름이 뭔가요?', '홍길동입니다.'],
 ['환자분 이름이 어떻게 되세요?', '홍길동입니다.'],
 ['어떻게 되시죠? 이름이?', '홍길동입니다.'],
 ['어떻게 되시죠? 성함이?', '홍길동입니다.'],
 ['이름이 어떻게 되시나요?', '홍길동입니다.'],
 ['성함이 어떻게 되시나요?', '홍길동입니다.'],
 ['이름이 무엇일까요?', '홍길동입니다.'],
 ['성함이 무엇일까요?', '홍길동입니다.'],
 ['성함 좀 말씀 해 주세요.', '홍길동입니다.'],
 ['이름 말씀해주실 수 있으실까요?', '홍길동입니다.'],
 ['성함 말씀해주실 수 있을실까요?', '홍길동입니다.'],
 ['이름 알려주실 수 있나요?', '홍길동입니다.']

In [10]:
length_list = []

for q in data_load:
    length_list.append(len(q[0]))

max_length = max(length_list)

In [11]:
max_length

195

In [12]:

os.chdir("/data/kmkim/chatbot_data_model")

def load_embedding():
    
    que_emb= torchtext.vocab.Vectors('FastText_hong_chatbot')

    
    que_vocab = torchtext.vocab.vocab(que_emb.stoi, min_freq= 0, specials = ["<unk>", "<pad>", "<sos>", "<eos>"])
    #equ_vocab = torchtext.vocab.vocab(equ_emb.stoi, min_freq= 0)
    
    vector_dic = {'que': que_emb} #{'que': que_emb, 'equ': equ_emb} # dictionary 형태로 해당 단어에 대한 벡터 값들 저장
    
    vocab_dic = {'que': que_vocab} #{'que' : que_vocab, 'equ' : equ_vocab} # 위와 같은 형태로 단어 사전 저장
    

    a = torch.zeros(4,256, requires_grad = True)                                 ## 해당 스페션 토큰에 대한 임베딩 값을 만들어주기 위함
    que_emb.vectors = torch.cat([a, que_emb.vectors], dim=0) #              concat 시켜 기존 벡터들에 스페셜 토큰의 임베딩 벡터를 만들어줌
    que_emb.stoi = dict(zip(que_emb.stoi.keys(), map(lambda x:x[1]+4, que_emb.stoi.items()))) # 스페셜 토큰이 추가 되었으니 기존 단어들의 인덱스를 뒤로 밀어줘야함
    for i, j in enumerate(["<unk>", "<pad>", "<sos>", "<eos>"]):
        que_emb.stoi[j] = i                  # 사전훈련으로 임베딩 된 단어들은 special 토큰이 없기에 삽입, torchtext 상위 버전에서는 따로 함수  제공
    que_vocab.set_default_index(0)                    #여기서 0번 인덱스는 <unk>를 의미함
    
    return que_vocab, que_emb

In [13]:
pre_vocab, pre_vector = load_embedding()

100%|██████████████████████████████████████████████████████████████████████████████████████████| 7490/7490 [00:00<00:00, 15197.36it/s]


In [133]:
len(pre_vocab)

7494

In [16]:
def for_yield_vocab_tokenizer(sentence): ## 문장을 형태소로 분리
    for i in sentence:
        tokens1 = mecab.morphs(i[0]) 
        tokens2 = mecab.morphs(i[1])
        total_tokens = tokens1 + tokens2
        yield total_tokens
        
def tokenizer(sentence):
    return mecab.morphs(sentence)

In [17]:
kor_vocab = build_vocab_from_iterator(for_yield_vocab_tokenizer(data_load), specials = ['<unk>', '<pad>', '<sos>', '<eos>'], min_freq = 1)
kor_vocab.set_default_index(kor_vocab['<unk>'])

In [18]:
len(kor_vocab)

7418

In [19]:
## Transformer에 적용되는 함수와 모델


class PositionalEncoding(nn.Module):
    def __init__(self, emb_size, max_len, dropout ):
        super(PositionalEncoding, self).__init__()
        
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size) ## shape = (1,256)
        pos = torch.arange(0, max_len).reshape(max_len, 1) ## shape (maxlen, 1)
        
        pos_embedding = torch.zeros((max_len, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)
        
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding) ## opimizer가 업데이트 하지 않음, 그러나 값은 존재, GPU연산 가능, 업데이트 하고싶지 않은 층
        
    def forward(self, token_embedding):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])
    

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super(TokenEmbedding,self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx = 1) ## 단어사전 
        self.emb_size = emb_size
    
    def forward(self, tokens):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)
    
    
class pre_TokenEmbedding(nn.Module): ## pretrain_word embedding
    def __init__(self, vocab, emb_size):
        super(pre_TokenEmbedding,self).__init__()
        self.embedding = nn.Embedding.from_pretrained(vocab.vectors, padding_idx=1, freeze= False)
        self.emb_size = emb_size
    def forward(self, tokens):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)
    

class Seq2SeqTransformer(nn.Module):
    def __init__(self, 
                 num_encoder_layers,
                 num_decoder_layers,
                 emb_size, 
                 nhead,
                 src_vocab_size,
                 tgt_vocab_size,
                 max_len,
                 dim_feedforward = 512, 
                 dropout = 0.4,
                pretrain = False):
        
        self.pretrain = pretrain
        super(Seq2SeqTransformer,self).__init__()
        
        self.transformer = Transformer(d_model = emb_size,
                                      nhead = nhead,
                                      num_encoder_layers = num_encoder_layers,
                                      num_decoder_layers = num_decoder_layers,
                                      dim_feedforward = dim_feedforward, dropout = dropout)
        
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        
        if pretrain == True:
            self.src_tok_emb = pre_TokenEmbedding(pre_vector, emb_size)
            self.tgt_tok_emb = pre_TokenEmbedding(pre_vector, emb_size)

        else:
            self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size) ##pre_TokenEmbedding TokenEmbedding(src_vocab_size, emb_size)   # pre_TokenEmbedding(pre_emb['que'], emb_size)
            self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)

            
        self.positional_encoding = PositionalEncoding(emb_size, max_len, dropout = dropout)
    
    def forward(self, src, tgt, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(tgt))
        
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        
        return self.generator(outs)
        
    def encode(self, src, src_mask):
        return self.transformer.encoder(self.positional_encoding(self.src_tok_emb(src)),
                                       src_mask)
    
    def decoder(self,tgt, memory, tgt_mask):
        return self.transformer.decoder(self.positional_encoding(self.tgt_tok_emb(tgt)),
                                       memory, tgt_mask)
    
    def ck_pretrain(self):
        return self.pretrain

In [20]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz,sz), device = device ))== 1).T  ## 삼각행렬로 만든 뒤
    mask = mask.float().masked_fill(mask==0, float('-inf')).masked_fill(mask == 1, float(0.0)) ## 0은 -inf 와 같은 작은 값을 줘서 
    return mask

def create_mask(src,tgt):
    src_seq_len = src.shape[0]   ## sequence_length
    tgt_seq_len = tgt.shape[0]   ## sequence_length
    
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len) ## 디코더에서 셀프 어텐션 할때 그 뒤를 못보게 마스킹
    src_mask = torch.zeros((src_seq_len, src_seq_len), device = device).type(torch.bool)
    
    src_padding_mask = (src == 1).T ## (33,128)이 input으로 들어오는데 왜 transpose를 하는지=> 패딩이 위쪽
    tgt_padding_mask = (tgt == 1).T
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [21]:
SRC_VOCAB_SIZE = len(pre_vocab) ## vocab_transform[SRC_LANGUAGE] pre_vocab[SRC_LANGUAGE]
TGT_VOCAB_SIZE = len(pre_vocab) ## vocab_transform[TGT_LANGUAGE] pre_vocab[TGT_LANGUAGE]
EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 4 ## 3  
NUM_DECODER_LAYERS = 4 ## 3


In [22]:
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, max_length, FFN_HID_DIM,
                                 pretrain = True)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index = 1)

optimizer = torch.optim.Adam(transformer.parameters(), lr = 5e-4, betas = (0.9,0.98), eps = 1e-9) ## lr = 4e-4 

In [23]:
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def tensor_transform(token_ids):
    return torch.cat((torch.tensor([2]),
                     torch.tensor(token_ids),
                     torch.tensor([3])))
text_transform = {}


text_transform['kor'] = sequential_transforms(tokenizer, pre_vocab, tensor_transform) ## <tokenizing, indexing, special token 추가> 식으로 진행
## text_transform['kor'] = sequential_transforms(tokenizer, pre_vocab, tensor_transform)


In [24]:
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform['kor'](src_sample))
        tgt_batch.append(text_transform['kor'](tgt_sample))

    
    src_batch = pad_sequence(src_batch, padding_value = 1.0)
    tgt_batch = pad_sequence(tgt_batch, padding_value = 1.0)
    
    return src_batch, tgt_batch

In [109]:
denti_batch = 256
example_batch = 512-denti_batch



In [110]:
train_loader = DataLoader(data_load, batch_size = example_batch, shuffle = True, collate_fn = collate_fn, drop_last = True ,num_workers = 2)
train_loader2 = DataLoader(sample_load2, batch_size = denti_batch, shuffle = True, collate_fn= collate_fn, drop_last = True, num_workers = 2)

In [111]:
next(iter(train_loader))

(tensor([[  2,   2,   2,  ...,   2,   2,   2],
         [156, 106, 173,  ..., 366, 830, 327],
         [ 72,  63,  23,  ...,   7, 615,  20],
         ...,
         [  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1]]),
 tensor([[  2,   2,   2,  ...,   2,   2,   2],
         [  5,   5,   5,  ..., 299,   5,   5],
         [636,  30,  23,  ..., 301,  68,   4],
         ...,
         [  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1]]))

In [67]:
def train_epoch(model, optimizer, train_loader, train_loader2):
    '''
        Args:
            model (Module)
            optimizer (Optimizer)
            train_loader (Dataloader)
            valid_loader (Dataloader)
    '''
    model.train()
    losses = 0
    val_losses = 0

    for (src,tgt),(src2,tgt2) in zip(train_loader, cycle(train_loader2)):
        
        if src.size(0) > src2.size(0):
            src_num = src.size(0) - src2.size(0)
            src2 = torch.concat((src2, torch.ones(src_num,denti_batch, dtype = torch.int64)),0)
            src = torch.concat((src,src2), 1)
        else:
            src_num = src2.size(0) - src.size(0)
            src = torch.concat((src, torch.ones(src_num,example_batch, dtype = torch.int64)),0)
            src = torch.concat((src,src2), 1)

        if tgt.size(0) > tgt2.size(0):
            tgt_num = tgt.size(0) - tgt2.size(0)
            tgt2 = torch.concat((tgt2, torch.ones(tgt_num,denti_batch, dtype = torch.int64)),0)
            tgt = torch.concat((tgt,tgt2), 1)
        else:
            tgt_num = tgt2.size(0) - tgt.size(0)
            tgt = torch.concat((tgt, torch.ones(tgt_num,example_batch, dtype = torch.int64)),0)
            tgt = torch.concat((tgt,tgt2), 1) 
        src = src.to(device)
        tgt = tgt.to(device)
        tgt_input = tgt[:-1, :]
        
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        
        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)
        
        optimizer.zero_grad()
        
        tgt_out = tgt[1:, :]
        
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        
        optimizer.step()
        losses += loss.item()

    
    return losses / len(train_loader)


In [68]:
def new_greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)
    
    memory = model.encode(src, src_mask)
    ys = torch.ones(1,src.size(1)).fill_(start_symbol).type(torch.long).to(device) #ys = orch.tensor([[q_vocab['<sos>'] for i in range(src.size(1))]], device = device) ## 처음 시작은 <sos> 토큰
    for i in range(max_len):
        memory = memory.to(device)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0)))
        out = model.decoder(ys, memory, tgt_mask)
        out = out.transpose(0,1) # (sequence ,batch, hidden)  -> (batch,sequence ,hidden)
        prob = model.generator(out[:, -1]) ## 마지막 단어
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.unsqueeze(0)
        ys = torch.cat([ys, next_word], dim = 0)
    return ys

In [69]:
def translate_bleu(model,testset):
    total_bleu = 0
    acc = 0
    model.eval()
    with torch.no_grad():
        for loader_idx ,(src, tgt) in enumerate(testset):
            src = src.to(device)
            tgt = tgt.to(device)
            target_length = tgt.size(0)
            num_tokens = src.shape[0] ## sequnce_length
            
            src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool) 
            tgt_tokens = new_greedy_decode(model, src, src_mask, max_len = num_tokens, start_symbol=2) #.flatten()
 
            decoded_words = [] ## 예측 문장들을 담는 list
                                                        ## 예측 결과
            for batch_idx in range(src.size(1)):        ## 배치의 각 문장마다 돌면서 <eos> 토큰을 만나면 문장을 끊도록
                a = tgt_tokens[:, batch_idx]             ## idx 번째 배치의 문장 가져오기, a
                subwords = []                            ## 배치의 문장을 담을 list
                for sent_idx, k in enumerate(a):     
                    if a[sent_idx] == 3:                
                        subwords.append("<eos>")
                        subwords = (" ".join(subwords)).replace("<sos>", "").replace("<eos>","").replace("<pad>", "")
                        decoded_words.append(subwords)
                        break
                    else:
                        subwords.append(pre_vocab.lookup_token(a[sent_idx])) ## index to word 
                        if len(subwords) == (len(tgt_tokens)):
                            subwords.append("<eos>")
                            subwords = (" ".join(subwords)).replace("<sos>", "").replace("<eos>","").replace("<pad>", "")
                            decoded_words.append(subwords)
                            break 
                            
            #predict_tgt = " ".join(equ_vocab.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<sos>", "").replace("<eos>", "")            

            for pred_idx in range(src.size(1)): ## 각 시퀀스를 돌면서 계산
                predict_equ = decoded_words[pred_idx]
                real_equ = (" ".join(pre_vocab.lookup_tokens(tgt.T.tolist()[pred_idx]))).replace("<sos>", "").replace("<eos>","").replace("<pad>", "")
    
                
                decoded_words[pred_idx] = mecab.morphs(decoded_words[pred_idx])
                decoded_words[pred_idx].extend(["<eos>"])
                
                real = mecab.morphs(real_equ)
                real.extend(["<eos>"])
                
#                 print("question =",(' '.join(pre_vocab.lookup_tokens(src.T.tolist()[pred_idx]))).replace("<pad>", "").replace("<eos>",""))
#                 print('real =', ' '.join((real)))
#                 print('predicted=',  ' '.join(decoded_words[pred_idx]))
#                 print("-----------------------------------------")
                bleu = sentence_bleu([real], decoded_words[pred_idx],smoothing_function =None, weights = (0.4,0.3,0.3)) # SmoothingFunction().method6
                total_bleu += bleu   ## 배치의 bleu score 저장 나중에 총 배치 개수로 나눠 줘야함
                
        
    return total_bleu / len(testset.dataset)

In [28]:
os.chdir('/data/kmkim/chatbot_data_model/chatbot_experiments')
t = open('Fast_sentp_train_losses','w')
b = open('Fast_sentp_bleu_scores','w')
v = open('Fast_sentp_valid_losses','w')
a = open('Fast_sentpaccuracy','w')
t.close()
b.close()
v.close()
a.close()

In [112]:
import os
NUM_EPOCHS = 300 ## 50
best_val_loss = None
before_loss = 999999
count = 0
'''
train_losses = []
val_losses = []
bleu_scores = []
acc_list = []
'''
for epoch in range(1, 800):
    os.chdir('/data/kmkim/chatbot_data_model/chatbot_experiments')
    t = open('Fast_sentp_train_losses','a')
    b = open('Fast_bleu_scores','a')
    v = open('Fast_valid_losses','a')
    a = open('Fast_accuracy','a')
    try:
        start_time = timer()
        train_loss= train_epoch(transformer, optimizer, train_loader, train_loader2)
        end_time = timer()
        bleu_score = translate_bleu(transformer, train_loader2)
        #valid_end_time = timer()
        
        t.write(str(train_loss) + '\n')
        #b.write(str(bleu_score) + '\n')
        #v.write(str(val_loss) + '\n')
        #a.write(str(acc) + '\n')
    finally:
        
        
        
        
        
        
        t.close()
        b.close()
        v.close()
        a.close()
            
        
    print(f"EPOCH : {epoch}, Train Loss : {train_loss}, train_Bleu_Score :{bleu_score}, train_acc: {''} \
train_time : {(end_time - start_time):.3f}s,  valid_time :{''}s ") 
    
    before_loss = train_loss
    
    if not best_val_loss or train_loss < best_val_loss:
        os.chdir('/data/kmkim/chatbot_data_model/chatbot_experiments')
        if not os.path.isdir("Fast_sentp_transformer_save"):
            os.makedirs("Fast_sentp_transformer_save")
        
        torch.save(transformer.state_dict(), './transformer_save/Fast_hong.pt')
        print("model saved")
        best_val_loss = train_loss
        count = 0
        
    else:
        count += 1
        if count == 50:
            print("overfitting")
            break

/opt/conda/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


EPOCH : 1, Train Loss : 0.3294111052314744, train_Bleu_Score :0.7220611864820354, train_acc:  train_time : 49.049s,  valid_time :s 
model saved
EPOCH : 2, Train Loss : 0.3316346714059815, train_Bleu_Score :0.7244128703908143, train_acc:  train_time : 49.322s,  valid_time :s 
EPOCH : 3, Train Loss : 0.32886244641005535, train_Bleu_Score :0.7100695460289285, train_acc:  train_time : 50.186s,  valid_time :s 
model saved
EPOCH : 4, Train Loss : 0.3288201196048096, train_Bleu_Score :0.7036639356163198, train_acc:  train_time : 50.332s,  valid_time :s 
model saved
EPOCH : 5, Train Loss : 0.3269143632350077, train_Bleu_Score :0.7104057694244752, train_acc:  train_time : 49.790s,  valid_time :s 
model saved
EPOCH : 6, Train Loss : 0.32897467774744255, train_Bleu_Score :0.7153416035804476, train_acc:  train_time : 49.425s,  valid_time :s 
EPOCH : 7, Train Loss : 0.3298193215186359, train_Bleu_Score :0.7094997103266557, train_acc:  train_time : 49.516s,  valid_time :s 
EPOCH : 8, Train Loss : 0.

KeyboardInterrupt: 

In [ ]:
ls

In [113]:
os.chdir('./transformer_save')

In [134]:
transformer_T = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, max_length, FFN_HID_DIM,
                                 pretrain = True)
transformer_T = transformer_T.to(device)
transformer_T.load_state_dict(torch.load('Fast_hong.pt')) #Fast_sentp_transformer_base_renew.pt

<All keys matched successfully>

In [135]:
def new_new_greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)
    
    memory = model.encode(src, src_mask)
    ys = torch.ones(1,src.size(1)).fill_(start_symbol).type(torch.long).to(device) #ys = orch.tensor([[q_vocab['<sos>'] for i in range(src.size(1))]], device = device) ## 처음 시작은 <sos> 토큰
    for i in range(max_len):
        memory = memory.to(device)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0)))
        out = model.decoder(ys, memory, tgt_mask)
        out = out.transpose(0,1) # (sequence ,batch, hidden)  -> (batch,sequence ,hidden)
        prob = model.generator(out[:, -1]) ## 마지막 단어
        _, next_word = torch.max(prob, dim = 1)
        if next_word == 3:
            break
        next_word = next_word.unsqueeze(0)
        ys = torch.cat([ys, next_word], dim = 0)
    return ys

In [136]:
def translate(model, src_sentence):
    model.eval()
    src = text_transform['kor'](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = new_new_greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 4, start_symbol=2).flatten()
    return (" ".join(pre_vocab.lookup_tokens(tgt_tokens.tolist()))).replace("<sos>", "").replace("<eos>","").replace("<pad>", "") # " ".join(kor_vocab.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<sos>", "").replace("<eos>", "") ## vocab.DecodeIds(tgt_tokens)

In [137]:
import re
k = translate(transformer_T, "성함이요?") ##AI hub ㅇㅇㅇ입니다
answer = re.sub('\ㅇ', "홍길동", k.strip())
print(answer)

홍길동이요 .


In [125]:
soy_fname = '/data/kmkim/chatbot_data_model/soy_correct.model'
soy_model = CountSpace()
soy_model.load_model(soy_fname, json_format= False)

In [126]:
soy_model.correct('무슨일이신가요?')

('무슨일이신가요?', [None, None, None, None, None, 0, 0, 1])

In [43]:
def chatbot_bleu(model,testset):
    total_bleu = 0
    acc = 0
    model.eval()
    with torch.no_grad():
        for loader_idx ,(src, tgt) in enumerate(testset):
            src = src.to(device)
            tgt = tgt.to(device)
            target_length = tgt.size(0)
            num_tokens = src.shape[0] ## sequnce_length
            
            src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool) 
            tgt_tokens = new_greedy_decode(model, src, src_mask, max_len = num_tokens, start_symbol=2) #.flatten()
 
            decoded_words = [] ## 예측 문장들을 담는 list
                                                        ## 예측 결과
            for batch_idx in range(src.size(1)):        ## 배치의 각 문장마다 돌면서 <eos> 토큰을 만나면 문장을 끊도록
                a = tgt_tokens[:, batch_idx]             ## idx 번째 배치의 문장 가져오기, a
                subwords = []                            ## 배치의 문장을 담을 list
                for sent_idx, k in enumerate(a):     
                    if a[sent_idx] == 3:                
                        subwords.append("<eos>")
                        subwords = (" ".join(subwords)).replace("<sos>", "").replace("<eos>","").replace("<pad>", "")
                        decoded_words.append(subwords)
                        break
                    else:
                        subwords.append(pre_vocab.lookup_token(a[sent_idx])) ## index to word 
                        if len(subwords) == (len(tgt_tokens)):
                            subwords.append("<eos>")
                            subwords = (" ".join(subwords)).replace("<sos>", "").replace("<eos>","").replace("<pad>", "")
                            decoded_words.append(subwords)
                            break 
                            
            #predict_tgt = " ".join(equ_vocab.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<sos>", "").replace("<eos>", "")            

            for pred_idx in range(src.size(1)): ## 각 시퀀스를 돌면서 계산
                predict_equ = decoded_words[pred_idx]
                real_equ = (" ".join(pre_vocab.lookup_tokens(tgt.T.tolist()[pred_idx]))).replace("<sos>", "").replace("<eos>","").replace("<pad>", "")
    
                
                decoded_words[pred_idx] = mecab.morphs(decoded_words[pred_idx])
                decoded_words[pred_idx].extend(["<eos>"])
                
                real = mecab.morphs(real_equ)
                real.extend(["<eos>"])
                print(pred_idx)
                print("question =",(' '.join(pre_vocab.lookup_tokens(src.T.tolist()[pred_idx]))).replace("<pad>", "").replace("<eos>",""))
                print('real =', ' '.join((real)))
                print('predicted=',  ' '.join(decoded_words[pred_idx]))
                print("-----------------------------------------")
                bleu = sentence_bleu([real], decoded_words[pred_idx],smoothing_function =None, weights= (0.4,0.3,0.3) )# SmoothingFunction().method6
                total_bleu += bleu   ## 배치의 bleu score 저장 나중에 총 배치 개수로 나눠 줘야함
                
        
    return total_bleu / len(testset.dataset)

In [115]:
bleu_score = chatbot_bleu(transformer_T, train_loader2)
print(bleu_score)

0
question = <sos> 아프 셔서 왔 을까요 ?              
real = 오른쪽 아래 치아 가 시려서 왔 습니다 . <eos>
predicted= 오른쪽 아래 치아 가 시려서 왔 습니다 . <eos>
-----------------------------------------
1
question = <sos> 따로 복용 하 고 계시 는 약 은 없 으신 가요 ?       
real = 없 습니다 . <eos>
predicted= 없 습니다 . <eos>
-----------------------------------------
2
question = <sos> 뭘 해야 아팠 나요 ?              
real = 음식 을 먹 을 때 아팠 습니다 . <eos>
predicted= 음식 을 먹 을 때 아팠 습니다 . <eos>
-----------------------------------------
3
question = <sos> 신경 쓰였 던 치아 가 있 어서 오 셨 나요 ?        
real = 오른쪽 아래 치아 가 시려서 왔 습니다 . <eos>
predicted= 오른쪽 아래 치아 가 시려서 왔 습니다 . <eos>
-----------------------------------------
4
question = <sos> 아픔 이 있 을 때 어떻게 하 셨 나요 ?         
real = 진통제 를 먹 었 습니다 . <eos>
predicted= 진통제 를 먹 었 습니다 . <eos>
-----------------------------------------
5
question = <sos> 진료 받 기 위해서 왔 나요 ?            
real = 오른쪽 아래 치아 가 시려서 왔 습니다 . <eos>
predicted= 오른쪽 아래 치아 가 시려서 왔 습니다 . <eos>
-----------------------------------------
6
question = <sos> 불편 하 셔서 검사 받 으

0
question = <sos> 생활 중 불편 했 던 때 가 있 나요 ?      
real = 음식 을 먹 을 때 아팠 습니다 . <eos>
predicted= 음식 을 먹 을 때 아팠 습니다 . <eos>
-----------------------------------------
1
question = <sos> 어디 가 불편 해서 오 셨 을까요 ?        
real = 오른쪽 아래 치아 가 시려서 왔 습니다 . <eos>
predicted= 오른쪽 아래 치아 가 시려서 왔 습니다 . <eos>
-----------------------------------------
2
question = <sos> 아픔 을 줄어들 게 하 는 방법 이 있 었 나요 ?    
real = 그런 건 잘 모르 겠 어요 . <eos>
predicted= 그런 건 잘 모르 겠 어요 . <eos>
-----------------------------------------
3
question = <sos> 찬 음식 은 삼가해 주 세요 .         
real = 네 . 알 겠 습니다 . <eos>
predicted= 네 . 알 겠 습니다 . <eos>
-----------------------------------------
4
question = <sos> 먹 기 힘들 었 던 음식 이 있 나요 ?      
real = 찬 음식 을 먹 을 때 아팠 습니다 . <eos>
predicted= 찬 음식 을 먹 을 때 아팠 습니다 . <eos>
-----------------------------------------
5
question = <sos> 방사선 사진 확인 해 볼게요 .          
real = 네 . 알 겠 습니다 . <eos>
predicted= 네 . 알 겠 습니다 . <eos>
-----------------------------------------
6
question = <sos> 아팠 던 부위 가 있 어서 왔 을까요 ?       
real = 오

0
question = <sos> 아팠 던 부위 가 있 어서 왔 나요 ?        
real = 오른쪽 아래 치아 가 시려서 왔 습니다 . <eos>
predicted= 오른쪽 아래 치아 가 시려서 왔 습니다 . <eos>
-----------------------------------------
1
question = <sos> 아픈 정도 를 숫자 로 표현 해 주 세요 .       
real = 점수 로 하 면 6 점 정도 입니다 . <eos>
predicted= 점수 로 하 면 6 점 정도 입니다 . <eos>
-----------------------------------------
2
question = <sos> 또 아프 신 다른 곳 있 나요 ?         
real = 다른 곳 은 없 습니다 . <eos>
predicted= 다른 곳 은 없 습니다 . <eos>
-----------------------------------------
3
question = <sos> 최대한 빨리 치료 하 는 게 좋 겠 네요        
real = 내일 가능 할까요 ? <eos>
predicted= 내일 가능 할까요 ? <eos>
-----------------------------------------
4
question = <sos> 어떨 때 통증 이 없 어 졌 나요 ?        
real = 그런 건 잘 모르 겠 어요 . <eos>
predicted= 그런 건 잘 모르 겠 어요 . <eos>
-----------------------------------------
5
question = <sos> 아프 실 때 무엇 을 하 셨 죠 ?        
real = 진통제 를 먹 었 습니다 . <eos>
predicted= 진통제 를 먹 었 습니다 . <eos>
-----------------------------------------
6
question = <sos> 술 은 평소 에 많이 드 시 는 편 인가요 ?      
real = 달 에 8 

In [103]:
del train_loader2

In [127]:
!nvidia-smi

Thu Oct 20 12:18:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.76       Driver Version: 515.76       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:5E:00.0 Off |                  Off |
| 30%   25C    P8    20W / 230W |   2021MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [114]:
ls

Fast_sentp_transformer_base3.pt               output.mp3
Fast_sentp_transformer_base_renew.pt          output.wav
Fast_transformer_base.pt                      transformer_base.pt
google-cloud-cli-402.0.0-linux-x86_64.tar.gz


In [117]:
torch.save(transformer_T, 'total_model.pt')

In [118]:
ls

Fast_sentp_transformer_base3.pt               output.wav
Fast_sentp_transformer_base_renew.pt          total_model
Fast_transformer_base.pt                      total_model.pt
google-cloud-cli-402.0.0-linux-x86_64.tar.gz  transformer_base.pt
output.mp3


In [122]:
ex_model = torch.load('total_model.pt')

In [120]:
ls

Fast_sentp_transformer_base3.pt               output.mp3
Fast_sentp_transformer_base_renew.pt          output.wav
Fast_transformer_base.pt                      total_model.pt
google-cloud-cli-402.0.0-linux-x86_64.tar.gz  transformer_base.pt


In [117]:
import gc
gc.collect()
torch.cuda.empty_cache()